In [13]:
import pandas as pd 
import os
from glob import glob
import re 

# Import all control genotypes
folder_path = "/Users/rachel/Desktop/GitRepos/MachineLearning/RA_Risk/Controls"
file_list = glob(os.path.join(folder_path, "*.xls"))
all_dfs = []
for path in file_list:
    df = pd.read_excel(path)
    filename = os.path.basename(path)
    # print(filename)
    match = re.search(r'chr(\d+\d)', filename)
    if match:
        chromosome = match.group(1)
    else:
        chromosome = "X"
    df['chromosome'] = chromosome
    all_dfs.append(df)

CTRL_df = pd.concat(all_dfs, ignore_index=True)

In [14]:
# Control preprocessing
# CTRL_df["condition"] = "CTRL"
# CTRL_df['MAF'] = CTRL_df.apply(lambda row: min(row['allele_A_freq'], row['allele_B_freq']), axis=1)
CTRL_df = CTRL_df.rename(columns={'allele_A':'allele_A_CTRL', 'allele_B':'allele_B_CTRL'})
CTRL_df.head()

# We need allele counts! 
CTRL_df['allele_A_CTRL_Count'] = (CTRL_df['genotype_AA_count'] * 2) + (CTRL_df['genotype_AB_count'])
CTRL_df['allele_B_CTRL_Count'] = (CTRL_df['genotype_BB_count'] * 2) + (CTRL_df['genotype_AB_count'])

In [15]:
# Import RA genotypes
RA_df = pd.read_csv("/Users/rachel/Desktop/GitRepos/MachineLearning/RA_Risk/Disease_Genotypes_RA.txt", sep="\t", dtype={'chromosome':str,}, na_values=[".-000"])
print(RA_df.columns)

Index(['dbSNP_rsID', 'allele_A', 'allele_B', 'genotype_AA_count',
       'genotype_AB_count', 'genotype_BB_count', 'genotype_XX_count',
       'genotype_AA_freq', 'genotype_AB_freq', 'genotype_BB_freq',
       'allele_A_freq', 'allele_B_freq', 'allele_of_MAF', 'HWE_p_value',
       'genotype_suspicious', 'number_of_mapped', 'chromosome',
       'position_in_chromosome', 'duplicate_SNP', 'perlegen_rsID',
       'perlegen_allele_1', 'perlegen_allele_2', 'perlegen_chromosome',
       'perlegen_accession_id', 'perlegen_contig_position', 'perlegen_strand',
       'perlegen_Assayed_sequence'],
      dtype='object')


In [16]:
def complement(nucleotide):
    complement_dict = str.maketrans("ACTG", "TGAC")
    complement = nucleotide.translate(complement_dict)
    return(complement)

# RA preprocessing
RA_df['genotype_suspicious'] = RA_df['genotype_suspicious'].replace('*', 1).fillna(0).astype(int)
# RA_df['MAF'] = RA_df.apply(lambda row: min(row['allele_A_freq'], row['allele_B_freq']), axis=1)
# RA_df['condition'] = "RA"

# Strand mismatch between RA and CTRL samples found
# Computing complement to check for strand orientation mismatch
RA_df['allele_A_comp_RA'] = RA_df['allele_A'].apply(complement)
RA_df['allele_B_comp_RA'] = RA_df['allele_B'].apply(complement)
RA_df = RA_df.rename(columns={'allele_A':'allele_A_RA', 'allele_B':'allele_B_RA'})

# We need allele counts! 
RA_df['allele_A_RA_Count'] = (RA_df['genotype_AA_count'] * 2) + (RA_df['genotype_AB_count'])
RA_df['allele_B_RA_Count'] = (RA_df['genotype_BB_count'] * 2) + (RA_df['genotype_AB_count'])

RA_df.head()
RA_df.columns

/var/folders/2x/jkbf1_7146v0xqx_m99sdm000000gn/T/ipykernel_5682/2555743811.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  RA_df['genotype_suspicious'] = RA_df['genotype_suspicious'].replace('*', 1).fillna(0).astype(int)


Index(['dbSNP_rsID', 'allele_A_RA', 'allele_B_RA', 'genotype_AA_count',
       'genotype_AB_count', 'genotype_BB_count', 'genotype_XX_count',
       'genotype_AA_freq', 'genotype_AB_freq', 'genotype_BB_freq',
       'allele_A_freq', 'allele_B_freq', 'allele_of_MAF', 'HWE_p_value',
       'genotype_suspicious', 'number_of_mapped', 'chromosome',
       'position_in_chromosome', 'duplicate_SNP', 'perlegen_rsID',
       'perlegen_allele_1', 'perlegen_allele_2', 'perlegen_chromosome',
       'perlegen_accession_id', 'perlegen_contig_position', 'perlegen_strand',
       'perlegen_Assayed_sequence', 'allele_A_comp_RA', 'allele_B_comp_RA',
       'allele_A_RA_Count', 'allele_B_RA_Count'],
      dtype='object')

In [17]:
RA_subset = RA_df[['dbSNP_rsID','allele_A_RA', 'allele_B_RA', 'allele_A_comp_RA','allele_B_comp_RA','allele_A_RA_Count','allele_B_RA_Count']]
CTRL_subset = CTRL_df[['dbSNP_rsID','allele_A_CTRL','allele_B_CTRL','allele_A_CTRL_Count','allele_B_CTRL_Count']]
df_clean = pd.merge(RA_subset, CTRL_subset, how="inner", on="dbSNP_rsID")
df_clean.head()

front_cols = ['dbSNP_rsID', 'allele_A_RA', 'allele_B_RA', 'allele_A_comp_RA', 'allele_B_comp_RA', 'allele_A_CTRL', 'allele_B_CTRL']
other_cols = [col for col in df_clean.columns if col not in front_cols]
organized_df = df_clean[front_cols + other_cols]
organized_df.head()

,dbSNP_rsID,allele_A_RA,allele_B_RA,allele_A_comp_RA,allele_B_comp_RA,allele_A_CTRL,allele_B_CTRL,allele_A_RA_Count,allele_B_RA_Count,allele_A_CTRL_Count,allele_B_CTRL_Count
0,970,A,G,T,C,T,C,309,79,1441,427
1,2060,G,A,C,T,A,G,96,294,1411,457
2,2952,T,C,A,G,T,C,345,45,1614,254
3,3431,C,T,G,A,A,G,285,103,604,1264
4,3444,C,T,G,A,A,G,352,38,131,1737


In [18]:
def detect_mistmatch(row):
    disease_A = row['allele_A_RA']
    disease_B = row['allele_B_RA']
    disease_A_rev = row['allele_A_comp_RA']
    disease_B_rev = row['allele_B_comp_RA']
    ctrl_A = row['allele_A_CTRL']
    ctrl_B = row['allele_B_CTRL']
    
    # Case 1: alleles already match
    if disease_A == ctrl_A and disease_B == ctrl_B:
        return pd.Series(["direct"])
    
    # Case 2: alleles are flipped
    elif disease_A == ctrl_B and disease_B == ctrl_A:
        return pd.Series(["flipped"])
    
    # Case 3: Strand mismatch, direct
    elif disease_A_rev == ctrl_A and disease_B_rev == ctrl_B:
        return pd.Series(["rev comp direct"])
    
    # Case 4: Strand mismatch, flipped
    elif disease_A_rev == ctrl_B and disease_B_rev == ctrl_A:
        return pd.Series(["rev comp flipped"])
    
    # Case 5: no match (could be multiallelic, strand error, or data error)
    else:
        return pd.Series(["no match"])
    

df_clean[['match_type']] = (df_clean.apply(detect_mistmatch, axis=1))
#df_aligned = df_clean[df_clean['match_type']!= "no match"]
count = 0
for index, row in df_clean.iterrows(): 
    if row['match_type'] == "no match":
        count += 1
    else:
        pass

# print(count) # no discordant SNPs - all are matched as direct, flipped, or complementary alleles 


In [19]:
df_clean.head()

def harmonize_alleles(df):
    for index, row in df.iterrows():
        match = row['match_type']
        if match == "direct":
            pass
        if match == "flipped":
            df.at[index, 'allele_A_CTRL'], df.at[index, 'allele_B_CTRL'] = row['allele_B_CTRL'], row['allele_A_CTRL']
            df.at[index, 'allele_A_CTRL_Count'], df.at[index, 'allele_B_CTRL_Count'] = row['allele_B_CTRL_Count'], row['allele_A_CTRL_Count']
        elif match == "rev comp direct":
            df.at[index, 'allele_A_CTRL'], df.at[index, "allele_B_CTRL"] = row['allele_A_comp_RA'], row['allele_B_comp_RA']
        elif match == "rev comp flipped":
            df.at[index, 'allele_A_CTRL'], df.at[index, "allele_B_CTRL"] = row['allele_B_comp_RA'], row['allele_A_comp_RA']
            df.at[index, 'allele_A_CTRL_Count'], df.at[index, 'allele_B_CTRL_Count'] = row['allele_B_CTRL_Count'], row['allele_A_CTRL_Count']
        else:
            f"Match type not recognized: {index}, {row}, {match}"
    df['harmonized'] = "Yes"
    return df

df_harmonized = harmonize_alleles(df_clean)
df_harmonized.head()

,dbSNP_rsID,allele_A_RA,allele_B_RA,allele_A_comp_RA,allele_B_comp_RA,allele_A_RA_Count,allele_B_RA_Count,allele_A_CTRL,allele_B_CTRL,allele_A_CTRL_Count,allele_B_CTRL_Count,match_type,harmonized
0,970,A,G,T,C,309,79,T,C,1441,427,rev comp direct,Yes
1,2060,G,A,C,T,96,294,G,A,457,1411,flipped,Yes
2,2952,T,C,A,G,345,45,T,C,1614,254,direct,Yes
3,3431,C,T,G,A,285,103,A,G,1264,604,rev comp flipped,Yes
4,3444,C,T,G,A,352,38,A,G,1737,131,rev comp flipped,Yes


In [20]:
from scipy.stats import fisher_exact
import statsmodels
from statsmodels.stats.multitest import multipletests

def fisher_test(df):
    results = []
    for index, row in df.iterrows():
        table = [row['allele_A_RA_Count'], row['allele_B_RA_Count']], [row['allele_A_CTRL_Count'], row['allele_B_CTRL_Count']]
        oddr, pval = fisher_exact(table)
        results.append({
            'SNP_rsID': row['dbSNP_rsID'],
            'odds_ratio': oddr,
            'p_value':pval
        })
    fisher_df = pd.DataFrame(results)
    return fisher_df
        
fisher_df = fisher_test(df_harmonized)



In [21]:
fisher_df.head()
padj = []
for index, row in fisher_df.iterrows():
    p_adj = multipletests(row['p_value'], method='fdr_bh')[1]
    padj.append(p_adj)
fisher_df["p_adj"] = padj
fisher_df.head()
# Annotations for snps with p_adj of 0 
# df_filtered_00['SNP_rsID'] = 'rs' + df_filtered_00['SNP_rsID'].astype(str)
# df_00_sorted = df_filtered_00.sort_values(by='odds_ratio', ascending=False)
# df_00_sorted.head()


idx = df_harmonized[df_harmonized['dbSNP_rsID'] == 11227065].index[0]
print(idx)
df_harmonized.iloc[idx]


37301


dbSNP_rsID             11227065
allele_A_RA                   C
allele_B_RA                   A
allele_A_comp_RA              G
allele_B_comp_RA              T
allele_A_RA_Count           376
allele_B_RA_Count            10
allele_A_CTRL                 C
allele_B_CTRL                 A
allele_A_CTRL_Count        1857
allele_B_CTRL_Count          11
match_type              flipped
harmonized                  Yes
Name: 37301, dtype: object

In [22]:
from scipy.stats import chi2_contingency

def chi2_test(df):
    results = []
    for index, row in df.iterrows():
        table = [row['allele_A_RA_Count'], row['allele_B_RA_Count']],[row['allele_A_CTRL_Count'], row['allele_B_CTRL_Count']]
        chi2, p, dof, expected = chi2_contingency(table)
        results.append({
            'SNP_rsID': row['dbSNP_rsID'],
            'chi2': chi2,
            'p_value':p,
            'deg_freedom':dof,
            'expected':expected
        })
    chi2_df = pd.DataFrame(results)
    return chi2_df

chi2_df = chi2_test(df_harmonized)

padj = []
for index, row in chi2_df.iterrows():
    p_adj = multipletests(row['p_value'], method='fdr_bh')[1]
    padj.append(p_adj)
chi2_df["p_adj"] = padj
chi2_df.head()


,SNP_rsID,chi2,p_value,deg_freedom,expected,p_adj
0,970,1.013008,0.314183,1,"[[300.9751773049645, 87.02482269503547], [1449...",[0.31418340906441816]
1,2060,0.000000,1.000000,1,"[[95.51372896368467, 294.4862710363153], [457....",[1.0]
2,2952,1.018105,0.312969,1,"[[338.35695305580157, 51.6430469441984], [1620...",[0.3129690377964257]
3,3431,4.735934,0.029539,1,"[[266.40602836879435, 121.59397163120568], [12...",[0.029538800542831384]
4,3444,3.091395,0.078707,1,"[[360.81045172719223, 29.189548272807794], [17...",[0.07870732241078082]


In [109]:
filtered = chi2_df[chi2_df['p_adj']<0.05]
len(filtered)
filtered_test_df = filtered.head(n=40)

In [ ]:
# Converting SNP ids to rs IDs for NCBI search
snp_list = filtered_test_df['SNP_rsID'].astype(str).tolist()
rs_ids = []
for snp in snp_list:
    rs_id = "rs" + snp
    rs_ids.append(rs_id)


# API Calls to retrieve data for each SNP
import requests
import re
import time

# 1. Create batches of 100 
def create_batch(iterable, n=20):
    for i in range(0, len(iterable), n):
        yield iterable[i:i+n]

# Convert rsID to UID
def rs_to_uid(snp_rsID_list):
    db = "snp"
    base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    term = " OR ".join(snp_rsID_list)
    parameters = {
        "db":db,
        "term": term,
        "retmode":"xml"
    }
    url = f"{base}esearch.fcgi"
    response = requests.get(url, parameters)
    response.raise_for_status()
    with open("snp_uid.xml", "a") as file:
        file.write(response.text)
  
for chunk in create_batch(rs_ids, n=20):
    uids = rs_to_uid(chunk)
    time.sleep(1)
    

In [139]:
from bs4 import BeautifulSoup
import lxml
import re
uid_query_list = []
with open("snp_uid.xml", "r") as file: 
    contents = file.read()
    clean_contents = re.sub(r'<\?xml.*?\?>', "", contents)
    clean_contents = re.sub(r'<!DOCTYPE.*?>', "", clean_contents)
    wrapped_contents = f"<root>{clean_contents}</root>"
    soup = BeautifulSoup(wrapped_contents, features="xml")
    id_list = soup.find_all("Id")
    for id in id_list: 
        uid_query_list.append(id.text)

print(uid_query_list)


['386605925', '386575067', '386431412', '111190448', '61074242', '60993869', '60931066', '60802411', '60666408', '60501608', '60352433', '60331063', '60030157', '59783138', '58976013', '58870618', '58655838', '58339502', '58159257', '57834999', '2100230641', '2100230631', '2100230624', '2100230612', '386624444', '386623847', '386623308', '386606604', '386606419', '61511205', '61506800', '60996781', '60551916', '60244893', '60071076', '59216045', '58844537', '58697252', '58667553', '58617118']


In [145]:
def snpdb_call(uid_list):
    """ 
    API call to retrieve data for all SNPs in the given list.
    :param snp_rsID_list: list of all SNPs to query. SNPs must be preceeded by rs (example: rs3431)
    """
    db = "snp"
    base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"

    # POST request
    post_data = {
        "db": db,
        "ids":uid_list
    }

    epost_url = f"{base}epost.fcgi"
    epost = requests.post(epost_url, post_data)
    return epost.text

    # if epost.status_code != 200:
    #     raise Exception(f"Epost request failed. Status code: {epost.status_code}")
    
    # web_match = re.search(r"<WebEnv>(\S+)</WebEnv>", str(epost))
    # web = web_match.group(1) if web_match else None
    # if web == None:
    #     raise Exception("Web value not found.")
    
    # key_match = re.search(r"<QueryKey>(\d+)</QueryKey>", str(epost))
    # key = key_match.group(1) if key_match else None
    # if key == None: 
    #     raise Exception("Key value not found.")    
    
    # efetch_url = f"{base}efetch.fcgi?db={db}&query_key={key}&WebEnv={web}&retmode=xml"
    # efetch = requests.get(efetch_url)

    # if efetch.status_code != 200:
    #     raise Exception(f"Efetch request failed. Status code: {efetch.status_code}")
    
    # return efetch.text

snpdb_call(uid_query_list)

'<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE ePostResult PUBLIC "-//NLM//DTD epost 20090526//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20090526/epost.dtd"><ePostResult>\n\t<ERROR>Empty ID list; Nothing to store</ERROR>\n</ePostResult>\n'